200
application/json
{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.32.5", 
    "X-Amzn-Trace-Id": "Root=1-6971e14f-12b520d02720c0f659672c50"
  }, 
  "origin": "89.180.58.25", 
  "url": "https://httpbin.org/get"
}

{'args': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.32.5', 'X-Amzn-Trace-Id': 'Root=1-6971e14f-12b520d02720c0f659672c50'}, 'origin': '89.180.58.25', 'url': 'https://httpbin.org/get'}


In [3]:
params = {"q": "python", "page": 2}
r = requests.get("https://httpbin.org/get", params=params, timeout=10)
print(r.url)  # final URL with querystring


https://httpbin.org/get?q=python&page=2


In [4]:
headers = {
    "Accept": "application/json",
    "User-Agent": "my-app/1.0",
}
r = requests.get("https://httpbin.org/headers", headers=headers, timeout=10)
print(r.json())


{'headers': {'Accept': 'application/json', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'my-app/1.0', 'X-Amzn-Trace-Id': 'Root=1-6971e178-6fd8e4c438e554ce417f3cd7'}}


In [5]:
payload = {"name": "Diogo", "role": "platform"}
r = requests.post("https://httpbin.org/post", json=payload, timeout=10)
r.raise_for_status()
print(r.json())


{'args': {}, 'data': '{"name": "Diogo", "role": "platform"}', 'files': {}, 'form': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Content-Length': '37', 'Content-Type': 'application/json', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.32.5', 'X-Amzn-Trace-Id': 'Root=1-6971e179-4819a0323aafe0b44a55e09a'}, 'json': {'name': 'Diogo', 'role': 'platform'}, 'origin': '89.180.58.25', 'url': 'https://httpbin.org/post'}


In [6]:
data = {"username": "user", "password": "secret"}
r = requests.post("https://httpbin.org/post", data=data, timeout=10)


In [12]:
import requests

try:
    r = requests.get("https://httpbin.org/get", timeout=10)
    r.raise_for_status()
    data = r.json()
except requests.exceptions.Timeout:
    print("Timed out")
except requests.exceptions.HTTPError as e:
    # You can inspect response body/status here
    print("HTTP error:", e, "status:", getattr(e.response, "status_code", None))
    print("body:", getattr(e.response, "text", "")[:500])
except requests.exceptions.RequestException as e:
    print("Request failed:", e)
else:
    print("ok:", data)


ok: {'args': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.32.5', 'X-Amzn-Trace-Id': 'Root=1-6971e225-54cabe0c64d9393a3103b2b6'}, 'origin': '89.180.58.25', 'url': 'https://httpbin.org/get'}


In [8]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

retry = Retry(
    total=5,
    backoff_factor=0.5,              # 0.5s, 1s, 2s, 4s...
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET", "POST", "PUT", "DELETE", "PATCH"],
    raise_on_status=False,
)

s = requests.Session()
s.mount("https://", HTTPAdapter(max_retries=retry))
s.mount("http://", HTTPAdapter(max_retries=retry))

r = s.get("https://httpbin.org/status/503", timeout=10)
print(r.status_code)


503
